### 4-4. KoNLPy 감정 분석 모델 학습

#### 데이터셋 준비

In [3]:
import pandas as pd
import os

train_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/naver_data/ratings_train.txt'
test_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/naver_data/ratings_test.txt'

# 데이터 불러오기
train_data = pd.read_csv(train_file, sep='\t')
test_data = pd.read_csv(test_file, sep='\t')

# 데이터 확인
print(train_data.head())
print(test_data.head())


         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [4]:
# 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
test_data.drop_duplicates(subset=['document'], inplace=True)

# 특수 문자 제거 (정규 표현식 사용)
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

print(f"전처리 후 학습 데이터 개수: {len(train_data)}")
print(f"전처리 후 테스트 데이터 개수: {len(test_data)}")

전처리 후 학습 데이터 개수: 146183
전처리 후 테스트 데이터 개수: 49158


In [5]:
# 결측값(NaN) 제거
train_data = train_data.dropna(subset=['document']).copy()
test_data = test_data.dropna(subset=['document']).copy()

# 문장 길이 필터링 (NaN 방지)
filtered_train_data = train_data[train_data['document'].apply(lambda x: 1 <= len(str(x)) <= 140)].copy()
filtered_test_data = test_data[test_data['document'].apply(lambda x: 1 <= len(str(x)) <= 140)].copy()

print(f"필터링 후 학습 데이터 개수: {len(filtered_train_data)}")
print(f"필터링 후 테스트 데이터 개수: {len(filtered_test_data)}")


필터링 후 학습 데이터 개수: 145791
필터링 후 테스트 데이터 개수: 48995


#### Mecab

In [6]:
from konlpy.tag import Mecab

# 형태소 분석기 변경
tokenizer = Mecab()

def tokenize_konlpy(corpus, tokenizer):
    return [' '.join(tokenizer.morphs(sentence)) for sentence in corpus]

# KoNLPy 기반 토큰화 적용 (Mecab 사용)
train_corpus_konlpy = tokenize_konlpy(filtered_train_data['document'], tokenizer)
test_corpus_konlpy = tokenize_konlpy(filtered_test_data['document'], tokenizer)

print("KoNLPy (Mecab) 토큰화 예시:", train_corpus_konlpy[:5])


KoNLPy (Mecab) 토큰화 예시: ['아 더 빙 진짜 짜증 나 네요 목소리', '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나', '너무 재 밓었다그래서보는것을추천한다', '교도소 이야기 구먼 솔직히 재미 는 없 다 평점 조정', '사이몬페그 의 익살 스런 연기 가 돋보였 던 영화 스파이더맨 에서 늙 어 보이 기 만 했 던 커스틴 던스트 가 너무나 도 이뻐 보였 다']


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 토크나이저 정의
konlpy_tokenizer = Tokenizer()
konlpy_tokenizer.fit_on_texts(train_corpus_konlpy)

# 정수 인코딩
train_tensor_konlpy = konlpy_tokenizer.texts_to_sequences(train_corpus_konlpy)
test_tensor_konlpy = konlpy_tokenizer.texts_to_sequences(test_corpus_konlpy)

# 패딩 적용
train_tensor_konlpy = pad_sequences(train_tensor_konlpy, padding='post')
test_tensor_konlpy = pad_sequences(test_tensor_konlpy, padding='post')

print("KoNLPy 기반 변환된 훈련 데이터 크기:", train_tensor_konlpy.shape)
print("KoNLPy 기반 변환된 테스트 데이터 크기:", test_tensor_konlpy.shape)


KoNLPy 기반 변환된 훈련 데이터 크기: (145791, 95)
KoNLPy 기반 변환된 테스트 데이터 크기: (48995, 93)


In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 같은 모델 구조로 학습
konlpy_model = Sequential([
    Embedding(input_dim=len(konlpy_tokenizer.word_index) + 1, output_dim=128, mask_zero=True),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

konlpy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 레이블 데이터 준비
train_labels = filtered_train_data['label'].values
test_labels = filtered_test_data['label'].values

# 모델 학습
konlpy_model.fit(train_tensor_konlpy, train_labels, 
                 validation_data=(test_tensor_konlpy, test_labels),
                 epochs=5, batch_size=64)


Epoch 1/5
2278/2278 [==============================] - 138s 58ms/step - loss: 0.3696 - accuracy: 0.8345 - val_loss: 0.3306 - val_accuracy: 0.8558
Epoch 2/5
2278/2278 [==============================] - 131s 57ms/step - loss: 0.2637 - accuracy: 0.8901 - val_loss: 0.3275 - val_accuracy: 0.8578
Epoch 3/5
2278/2278 [==============================] - 129s 56ms/step - loss: 0.2023 - accuracy: 0.9182 - val_loss: 0.3503 - val_accuracy: 0.8580
Epoch 4/5
2278/2278 [==============================] - 130s 57ms/step - loss: 0.1558 - accuracy: 0.9392 - val_loss: 0.3825 - val_accuracy: 0.8551
Epoch 5/5
2278/2278 [==============================] - 131s 58ms/step - loss: 0.1240 - accuracy: 0.9532 - val_loss: 0.4483 - val_accuracy: 0.8516


In [13]:
konlpy_test_loss, konlpy_test_acc = konlpy_model.evaluate(test_tensor_konlpy, test_labels)
print(f"KoNLPy 기반 테스트 정확도: {konlpy_test_acc:.4f}")


1532/1532 [==============================] - 8s 5ms/step - loss: 0.4483 - accuracy: 0.8516
KoNLPy 기반 테스트 정확도: 0.8516


#### SentencePiece VS Mecab
테스트 정확도   
0.8527 > 0.8516

#### Komoran

In [15]:
# NaN 값 및 빈 문자열 제거
filtered_train_data = filtered_train_data.dropna(subset=['document'])
filtered_test_data = filtered_test_data.dropna(subset=['document'])

filtered_train_data = filtered_train_data[filtered_train_data['document'].str.strip() != '']
filtered_test_data = filtered_test_data[filtered_test_data['document'].str.strip() != '']


In [17]:
from konlpy.tag import Komoran

# 형태소 분석기 변경 (Komoran)
tokenizer = Komoran()

def tokenize_konlpy(corpus, tokenizer):
    return [' '.join(tokenizer.morphs(str(sentence))) for sentence in corpus]

# 형태소 분석기 변경 (Komoran)
from konlpy.tag import Komoran
tokenizer = Komoran()

# KoNLPy 기반 토큰화 적용 (Komoran 사용)
train_corpus_konlpy = tokenize_konlpy(filtered_train_data['document'], tokenizer)
test_corpus_konlpy = tokenize_konlpy(filtered_test_data['document'], tokenizer)

print("KoNLPy (Komoran) 토큰화 예시:", train_corpus_konlpy[:5])


KoNLPy (Komoran) 토큰화 예시: ['아 더빙 진짜 짜증 나 네요 목소리', '흠 포스터 보고 초딩 영화 줄 오버 연기 조차 가볍 지 않 구나', '너무재밓었다그래서보는것을추천한다', '교도소 이야기 이 구먼 솔직히 재미 는 없 다 평점 조정', '사이몬페그의 익살 스럽 ㄴ 연기 가 돋보이 었 던 영화 스파이더맨 에서 늙 어 보이 기 만 하 았 던 커스틴 던스트 가 너무나 도 이뻐보였다']


In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 토크나이저 정의
konlpy_tokenizer = Tokenizer()
konlpy_tokenizer.fit_on_texts(train_corpus_konlpy)

# 정수 인코딩
train_tensor_konlpy = konlpy_tokenizer.texts_to_sequences(train_corpus_konlpy)
test_tensor_konlpy = konlpy_tokenizer.texts_to_sequences(test_corpus_konlpy)

# 패딩 적용
train_tensor_konlpy = pad_sequences(train_tensor_konlpy, padding='post')
test_tensor_konlpy = pad_sequences(test_tensor_konlpy, padding='post')

print("KoNLPy 기반 변환된 훈련 데이터 크기:", train_tensor_konlpy.shape)
print("KoNLPy 기반 변환된 테스트 데이터 크기:", test_tensor_konlpy.shape)

KoNLPy 기반 변환된 훈련 데이터 크기: (145393, 104)
KoNLPy 기반 변환된 테스트 데이터 크기: (48852, 105)


In [19]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 같은 모델 구조로 학습
konlpy_model = Sequential([
    Embedding(input_dim=len(konlpy_tokenizer.word_index) + 1, output_dim=128, mask_zero=True),
    LSTM(64),
    Dense(1, activation='sigmoid')
])

konlpy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 레이블 데이터 준비
train_labels = filtered_train_data['label'].values
test_labels = filtered_test_data['label'].values

# 모델 학습
konlpy_model.fit(train_tensor_konlpy, train_labels, 
                 validation_data=(test_tensor_konlpy, test_labels),
                 epochs=5, batch_size=64)


Epoch 1/5
2272/2272 [==============================] - 74s 28ms/step - loss: 0.3877 - accuracy: 0.8216 - val_loss: 0.3497 - val_accuracy: 0.8447
Epoch 2/5
2272/2272 [==============================] - 49s 21ms/step - loss: 0.2796 - accuracy: 0.8850 - val_loss: 0.3418 - val_accuracy: 0.8491
Epoch 3/5
2272/2272 [==============================] - 38s 17ms/step - loss: 0.2172 - accuracy: 0.9131 - val_loss: 0.3606 - val_accuracy: 0.8484
Epoch 4/5
2272/2272 [==============================] - 52s 23ms/step - loss: 0.1700 - accuracy: 0.9326 - val_loss: 0.3922 - val_accuracy: 0.8485
Epoch 5/5
2272/2272 [==============================] - 53s 23ms/step - loss: 0.1355 - accuracy: 0.9473 - val_loss: 0.4276 - val_accuracy: 0.8428


In [20]:
konlpy_test_loss, konlpy_test_acc = konlpy_model.evaluate(test_tensor_konlpy, test_labels)
print(f"KoNLPy 기반 테스트 정확도: {konlpy_test_acc:.4f}")


1527/1527 [==============================] - 22s 15ms/step - loss: 0.4276 - accuracy: 0.8428
KoNLPy 기반 테스트 정확도: 0.8428


#### SentencePiece VS Komoran
테스트 정확도   
0.8527 > 0.8428